# Hydro Power Plant Optimisation

This Notebook aims at defining the optimisation of a Hydrow Power Plant

We recall the different variables:
    
- $q(t)$ the flow that is ran
- $S(t)$ the power spot price
- $X(t)$ the level of the reservoir
- $a(t)$ the inflow of water to the reservoir
- $\delta(t)$ the spilling, i.e. the water that leaves the reservoir if the production is at its maximum
- $q_\text{min}(t)$ and $q_\text{max}(t)$ the daily bounds on flown quantities
- $X_\text{min}(t)$ the daily minimum of water in the reservoir
- $X_\text{end}$ the final water level minimum in reservoir

In [1]:
import numpy as np
import scipy.optimize as sco

In [2]:
def objective(q):
    """
    Returns the payoff of selling q for price s 
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - the quantity of water flown in m^3
    s, array - prices in €/MWH
    h, float - head (ie altitude of the water fall) in m
    rho, float - density of water, = 1000kg/m^3
    g , float - gravitational constant, = 9.81m/s^2
    cf, float - conversion factor from Joule to MWH, =2.78 e-7
    
    """
    cf = 2.78e-7
    g = 9.81
    rho = 1000
    
    return -sum([x*y*h*rho*g*cf for x,y in zip(*[q,s])])

In [3]:


def cons_periodicflow_min(q):
    """
    Constraint on minimum flow during the period
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - quantity of water flown
    
    """
    
    return q.sum()-qpmin

def cons_periodicflow_max(q):
    """
    Constraint on minimum flow during the period
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - quantity of water flown
    
    """
    return qpmax - q.sum()

def cons_daily_reservoir_max(q):
    """
    Constraint on daily maximum water level in reservoir
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - quantity of water flown
    
    """
    
    return np.array([x0+np.sum(a[:t+1])-np.sum(q[:t+1]) +xmax[t] for t in range(0,T)])

def cons_daily_reservoir_min(q):
    """
    Constraint on daily minimum water level in reservoir
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - quantity of water flown
    
    """
    
    return np.array([x0+np.sum(a[:t+1])-np.sum(q[:t+1]) -xmin[t] for t in range(0,T)])

con1 = {'type':'ineq','fun':cons_periodicflow_min}
con2 = {'type':'ineq','fun':cons_periodicflow_max}
con3 = {'type':'ineq','fun':cons_daily_reservoir_min}
con4 = {'type':'ineq','fun':cons_daily_reservoir_max}

cons=[con1,con2,con3,con4]



In [4]:

def HPP(q0,b,cons):
    """
    Optimizes the Hydro Power Plant (HPP)
    
    """
    sol = sco.minimize(objective,q0,bounds=b,constraints=cons)
    print("Payoff is :", -round(sol.fun,2))
    print("Optimal trajectory is :", [round(q,2) for q in sol.x])
    return sol

In [7]:
# parameters
a = [10 for i in range(50)]                          # inflows in the reservoir
T = len(a)                                   # number of days
q0 = np.zeros(T)                             # initial values for outflows
s = np.array([10 for i in range(T)])         # spot price
x0 = 100                                     # initial lvl of water in the reservoir
x_max = 1000                                  # reservoir water capacity
xmax = [x_max for i in range(T)]
h=0.85

# constraints        
qpmax = 150                                  # Max quantity of water flown out from the reservoir over the period
qpmin = 0                                    # Min quantity of water flown out from the reservoir over the period
xmin = [50 for i in range(T)]                # Daily minimum lvl of water in the reservoir

# bounds                 
qmin = [0 for i in range(T)]                 # daily minimum of water flown out from the reservoir
qmax = [50 for i in range(T)]                # daily minimum of water flown out from the reservoir
b = [(qmin[t],qmax[t]) for t in range(T)]    # bounds using the above

In [8]:
sol = sco.minimize(objective,q0,bounds=b,constraints=cons)
print("Payoff is :", -round(sol.fun,2))
print("Optimal trajectory is :", [round(q,2) for q in sol.x])

Payoff is : 3.48
Optimal trajectory is : [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]


In [9]:
HPP(q0,b,cons)

Payoff is : 3.48
Optimal trajectory is : [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]


     fun: -3.4771545000000024
     jac: array([-0.02318102, -0.02318102, -0.02318102, -0.02318102, -0.02318102,
       -0.02318102, -0.02318102, -0.02318102, -0.02318102, -0.02318102,
       -0.02318102, -0.02318102, -0.02318102, -0.02318102, -0.02318102,
       -0.02318102, -0.02318102, -0.02318102, -0.02318102, -0.02318102,
       -0.02318102, -0.02318102, -0.02318102, -0.02318102, -0.02318102,
       -0.02318102, -0.02318102, -0.02318102, -0.02318102, -0.02318102,
       -0.02318105, -0.02318102, -0.02318102, -0.02318102, -0.02318102,
       -0.02318102, -0.02318102, -0.02318102, -0.02318102, -0.02318102,
       -0.02318102, -0.02318102, -0.02318102, -0.02318102, -0.02318102,
       -0.02318102, -0.02318102, -0.02318102, -0.02318102, -0.02318105])
 message: 'Optimization terminated successfully.'
    nfev: 260
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([3.00000004, 3.00000004, 3.00000004, 3.00000004, 3.00000004,
       3.00000004, 3.00000004, 3.00000004, 2.99